In [1]:
# https://arxiv.org/abs/1610.02415

# https://pytorch-geometric.readthedocs.io/en/latest/notes/introduction.html

import torch
print(torch.__version__)
import torch.nn.functional as F
import torch.nn as nn
import torch.distributed as dist

import torch_geometric
import torch_geometric.nn as gnn
#print(torch_geometric.__version__)
from torch_geometric.datasets import QM9
import GCL.augmentors
import GCL.augmentors as A

from sklearn.model_selection import train_test_split

from sklearn.linear_model import RidgeClassifierCV, LogisticRegression, LinearRegression

1.13.1+cu117


In [2]:

parameters = {}
parameters['batch_size'] = 64

In [3]:
whole_dataset = QM9(root = 'data/')

#print(whole_dataset.get_summary())
#print(dir(whole_dataset))
#print(whole_dataset.len())

n = whole_dataset.len()
tr_n = 2000 # Number of QM9 to use as training data

all_inds = range(n)
tr_inds, val_inds = train_test_split(all_inds, train_size = tr_n)
print(len(tr_inds), len(val_inds))
print(type(tr_inds), type(tr_inds[0]))


train_sampler = torch.utils.data.SubsetRandomSampler(tr_inds)
val_sampler = torch.utils.data.SubsetRandomSampler(val_inds)

# We need to make a train and validation set since QM9 does not provide them
train_set = torch.utils.data.Subset(whole_dataset, tr_inds)
val_set = torch.utils.data.Subset(whole_dataset, val_inds)

train_loader = torch_geometric.loader.DataLoader(train_set, batch_size = parameters['batch_size'],
                                                shuffle = True, num_workers = 2,)
                                                #sampler = train_sampler)

val_loader = torch_geometric.loader.DataLoader(val_set, batch_size=2048,
                                            shuffle=False, num_workers=2,)
                                              #sampler = val_sampler)


2000 128831
<class 'list'> <class 'int'>


In [6]:
from torch_geometric.nn import GCNConv

class GCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        
        self.rep_dim = 128
        self.emb_dim = 256
        
        # Data under graph
        self.conv1 = GCNConv(whole_dataset.num_node_features, self.rep_dim // 2)
        self.bn1 = nn.BatchNorm1d(self.rep_dim // 2)
        self.a1 = nn.LeakyReLU(0.02)
        
        self.conv2 = GCNConv(self.rep_dim // 2, self.rep_dim) # To Rep Space
        self.bn2 = nn.BatchNorm1d(self.rep_dim)
        
        # Projection to representation
        self.mpool1 = gnn.global_mean_pool
        self.fc1 = nn.Linear(self.rep_dim, self.rep_dim)
        
        # Graph 2
        self.conv3 = GCNConv(self.rep_dim, self.rep_dim * 2) # To Emb Space
        self.bn3 = nn.BatchNorm1d(self.rep_dim * 2)
        
        # Projection to embedding
        self.mpool2 = gnn.global_mean_pool
        self.fc2 = nn.Linear(self.emb_dim, self.emb_dim) # Linear to rep?
        
    def forward(self, data, binds):
        x = data[0].float().to(device)
        edge_index = data[1].to(device)
        
        # Input graph to GConv
        x = self.conv1(x, edge_index)
        x = self.a1(self.bn1(x))
        x = F.dropout(x, training=self.training)
        
        x = self.bn2(self.conv2(x, edge_index))
        
        # GConv outputs projected to representation space
        print('before pool: ', x.shape)
        x_rep = self.mpool1(x, binds)
        print('pooled: ', x_rep.shape)
        
        x_rep = self.fc1(x_rep)
        print('projected: ', x_rep.shape, 'gconv', x.shape)
        
        x_emb = self.bn3(self.conv3(x, edge_index))
        print('x emb after conv3', x_emb.shape)
        x_emb = self.mpool2(x_emb, binds)
        print('after pool', x_emb.shape)
        x_emb = self.fc2(x_emb)
        print('after fc2', x_emb.shape)
        
        return x_rep, x_emb

device = 'cuda'

model = GCN().to(device)

aug = A.RandomChoice([#A.RWSampling(num_seeds=1000, walk_length=10),
                      A.NodeDropping(pn=0.1),
                      A.FeatureMasking(pf=0.1),
                      A.EdgeRemoving(pe=0.1)],
                     num_choices=1)

for batch in train_loader:

    batch_inds = batch.batch.to(device)
    
    # batch of graphs has edge attribs, node attribs - (n_nodes, n_features+1) -> concat (n_nodes, attrib1)
    
    batch.x = batch.x.float()#.to(device)
    #batch.edge_index = batch.edge_index.to(device)

    # Barlow - get 2 random views of batch
    b1 = aug(batch.x, batch.edge_index, batch.edge_attr)
    b2 = aug(batch.x, batch.edge_index, batch.edge_attr)
    
    # Embed each batch (ignoring representations)
    r1, e1 = model(b1, batch_inds)
    r2, e2 = model(b2, batch_inds)
    
    
    print('reps', r1.shape, r2.shape)
    print('embs', e1.shape, e2.shape)
    
    loss 
    
    
    if val_step:
        for batch in val_loader:
            
            rep, _ = model(val_aug(batch.x, batch.edge_index, batch.edge_attr), batch.batch.to(device))
            
            lm = LinearClassifier(rep, batch.y)
            
            for tar in tagets:
                lm = LinearClassifier(rep, batch.tar)
            

before pool:  torch.Size([1182, 128])
pooled:  torch.Size([64, 128])
projected:  torch.Size([64, 128]) gconv torch.Size([1182, 128])
x emb after conv3 torch.Size([1182, 256])
after pool torch.Size([64, 256])
after fc2 torch.Size([64, 256])
before pool:  torch.Size([1182, 128])
pooled:  torch.Size([64, 128])
projected:  torch.Size([64, 128]) gconv torch.Size([1182, 128])
x emb after conv3 torch.Size([1182, 256])
after pool torch.Size([64, 256])
after fc2 torch.Size([64, 256])
reps torch.Size([64, 128]) torch.Size([64, 128])
embs torch.Size([64, 256]) torch.Size([64, 256])
before pool:  torch.Size([1131, 128])
pooled:  torch.Size([64, 128])
projected:  torch.Size([64, 128]) gconv torch.Size([1131, 128])
x emb after conv3 torch.Size([1131, 256])
after pool torch.Size([64, 256])
after fc2 torch.Size([64, 256])
before pool:  torch.Size([1131, 128])
pooled:  torch.Size([64, 128])
projected:  torch.Size([64, 128]) gconv torch.Size([1131, 128])
x emb after conv3 torch.Size([1131, 256])
after p

x emb after conv3 torch.Size([1120, 256])
after pool torch.Size([64, 256])
after fc2 torch.Size([64, 256])
before pool:  torch.Size([1120, 128])
pooled:  torch.Size([64, 128])
projected:  torch.Size([64, 128]) gconv torch.Size([1120, 128])
x emb after conv3 torch.Size([1120, 256])
after pool torch.Size([64, 256])
after fc2 torch.Size([64, 256])
reps torch.Size([64, 128]) torch.Size([64, 128])
embs torch.Size([64, 256]) torch.Size([64, 256])
before pool:  torch.Size([1114, 128])
pooled:  torch.Size([64, 128])
projected:  torch.Size([64, 128]) gconv torch.Size([1114, 128])
x emb after conv3 torch.Size([1114, 256])
after pool torch.Size([64, 256])
after fc2 torch.Size([64, 256])
before pool:  torch.Size([1114, 128])
pooled:  torch.Size([64, 128])
projected:  torch.Size([64, 128]) gconv torch.Size([1114, 128])
x emb after conv3 torch.Size([1114, 256])
after pool torch.Size([64, 256])
after fc2 torch.Size([64, 256])
reps torch.Size([64, 128]) torch.Size([64, 128])
embs torch.Size([64, 256])

KeyboardInterrupt: 

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
model = GCN().to(device)
#data = train_dataset[0].to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)


aug = A.RandomChoice([#A.RWSampling(num_seeds=1000, walk_length=10),
                      A.NodeDropping(pn=0.1),
                      A.FeatureMasking(pf=0.1),
                      A.EdgeRemoving(pe=0.1)],
                     num_choices=1)

val_aug = A.RandomChoice([], num_choices = 0)


def barlow(batch):
    # Return two random views of input batch
    return aug(batch[0], batch[1]), aug(batch[0], batch[1])

def off_diagonal(x):
    n, m = x.shape
    assert n == m
    return x.flatten()[:-1].view(n - 1, n + 1)[:, 1:].flatten()

class FullGatherLayer(torch.autograd.Function):
    """
    Gather tensors from all process and support backward propagation
    for the gradients across processes.
    """

    @staticmethod
    def forward(ctx, x):
        output = [torch.zeros_like(x) for _ in range(dist.get_world_size())]
        dist.all_gather(output, x)
        return tuple(output)

    @staticmethod
    def backward(ctx, *grads):
        all_gradients = torch.stack(grads)
        dist.all_reduce(all_gradients)
        return all_gradients[dist.get_rank()]
    
def VicRegLoss(x, y):
    # https://github.com/facebookresearch/vicreg/blob/4e12602fd495af83efd1631fbe82523e6db092e0/main_vicreg.py#L184
    # x, y are output of projector(backbone(x and y))
    repr_loss = F.mse_loss(x, y)

    x = x - x.mean(dim=0)
    y = y - y.mean(dim=0)

    std_x = torch.sqrt(x.var(dim=0) + 0.0001)
    std_y = torch.sqrt(y.var(dim=0) + 0.0001)
    std_loss = torch.mean(F.relu(1 - std_x)) / 2 + torch.mean(F.relu(1 - std_y)) / 2

    cov_x = (x.T @ x) / (parameters['batch_size'] - 1)
    cov_y = (y.T @ y) / (parameters['batch_size'] - 1)
    cov_loss = off_diagonal(cov_x).pow_(2).sum().div(
        x.shape[1]
    ) + off_diagonal(cov_y).pow_(2).sum().div(x.shape[1])
    
    # self.num_features -> rep_dim?
    loss = (
        sim_coeff * repr_loss
        + std_coeff * std_loss
        + cov_coeff * cov_loss
    )
    return loss

sim_coeff = 25
std_coeff = 25
cov_coeff = 1

model.train()
for epoch in range(5):
    
    epo_losses = []
    for batch in train_loader:
        #batch = batch.to(device)
        batch.x = batch.x.float()#.to(device)
        #batch.edge_index = batch.edge_index.to(device)

        optimizer.zero_grad()
        
        # Barlow - get 2 random views of batch
        b1 = aug(batch.x, batch.edge_index, batch.edge_attr)
        b2 = aug(batch.x, batch.edge_index, batch.edge_attr)
        
                
        # Embed each batch (ignoring representations)
        [r1, e1], [r2, e2] = model.pair_emb_rep(b1, b2)

        # VicReg loss on projections
        loss = VicRegLoss(e1, e2)
        
        loss.backward()
        optimizer.step()
        
        epo_losses.append(loss.data.item())
        
    print(sum(epo_losses) / len(epo_losses))
    
    ############################
    ## Per-epoch validation step:
    
    GCL.eval


    # Embed Training Samples:
    train_batch = next(iter(train_big_subset))
    #print('train batch', train_batch)
    train_batch = val_aug(train_batch.x, train_batch.edge_index, train_batch.edge_attr) # val_aug is an empty augmentation
    #print('train_batch augd', train_batch)

    with torch.no_grad():
        tr_rep, _ = model.forward(train_batch)
    #print(tr_rep.shape)

    # Train linear model on embedded samples:
    ridge_mod = RidgeClassifierCV(cv = 4).fit(tr_rep, y_train)
    linear_mod = LogisticRegression(penalty = None).fit(tr_rep, y_train)

    # Embed validation samples:
    val_batch = next(iter(val_loader))
    #print('val batch', val_batch)
    val_batch = val_aug(val_batch.x, val_batch.edge_index, val_batch.edge_attr) # val_aug is an empty augmentation
    #print('val_batch augd', val_batch)
    
    with torch.no_grad():
        val_rep, _ = model.forward(val_batch)
    #print(val_rep.shape)

    # Test linear model on embedded samples:
    ridge_score = f1_score(ridge_mod.predict(val_rep), y_val)
    linear_score = f1_score(linear_mod.predict(val_rep), y_val)
    
    print(f'Classifier Scores at Epoch {epoch}:', round(linear_score, 3), round(ridge_score, 3))

cuda


KeyboardInterrupt: 

In [ ]:
if False: # Update for some downstream? Keep in mind this idea of graph masking
    # Evaluate
    model.eval()
    pred = model(data).argmax(dim=1)
    correct = (pred[data.test_mask] == data.y[data.test_mask]).sum()
    acc = int(correct) / int(data.test_mask.sum())
    print(f'Accuracy: {acc:.4f}')